In [1]:
!pip install onnx onnxruntime

In [2]:
import onnx
import onnxruntime as ort
import numpy as np
import math

# Define input and output tensor names
input1_name = "S"
input2_name = "L"
input3_name = "D"

range_output_name = "Y"


# Calculates the total number of elements of the inteneded range
def num_elements(S, L, D):
    if D == 0:
        raise ValueError("Step (D) cannot be zero.")
    return max(math.ceil((L - S) / D), 0)

# Create the ONNX model with Reshape operator
def create_reshape_model(out_shape, dtype):
    # Create "S" input tensor (scalar tensor)
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, [])
    # Create "L" input tensor (scalar tensor)
    input2 = onnx.helper.make_tensor_value_info(input2_name, dtype, [])
    # Create "D" input tensor (scalar tensor)   
    input3 = onnx.helper.make_tensor_value_info(input3_name, dtype, [])
    # Create output tensor (final result after range operation)
    range_output = onnx.helper.make_tensor_value_info(range_output_name, dtype, out_shape)

    # Define range node
    range_node = onnx.helper.make_node(
        "Range",
        inputs=[input1_name, input2_name, input3_name],
        outputs=[range_output_name],
    )

    # Create the ONNX graph (no output shape specified - will be inferred at runtime)
    graph_def = onnx.helper.make_graph(
        [range_node],
        "Range",
        [input1, input2, input3],
        [range_output],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 11)])
    model.ir_version = 10
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "range.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("range.onnx")
    return session

def do_range(S, L, D, session):
    # Run inference
    output = session.run(None, {input1_name: S, input2_name: L, input3_name: D})

    S_f = (np.array2string(S, separator=',', max_line_width=np.inf).replace('\n', ''))
    L_f = (np.array2string(L, separator=',', max_line_width=np.inf).replace('\n', ''))
    D_f = (np.array2string(D, separator=',', max_line_width=np.inf).replace('\n', ''))

    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"S shape: {S.shape}, X={S_f}")
    print(f"L shape: {L.shape}, X={L_f}")
    print(f"D shape: {D.shape}, X={D_f}")
    print(f"Output shape: {output[0].shape}, Result = {y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Simple cases

In [3]:
# Case N1 - ordered bounds and positive delta
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(0, dtype=np.int32)
L = np.array(10,  dtype=np.int32)
D = np.array(1, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

S shape: (), X=0
L shape: (), X=10
D shape: (), X=1
Output shape: (10,), Result = [0,1,2,3,4,5,6,7,8,9]


In [4]:
# Case N2 - ordered bounds and negative delta
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(0, dtype=np.int32)
L = np.array(10,  dtype=np.int32)
D = np.array(-1, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

S shape: (), X=0
L shape: (), X=10
D shape: (), X=-1
Output shape: (0,), Result = []


In [5]:
# Case N3 - inverted bounds and positive delta
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(10, dtype=np.int32)
L = np.array(0,  dtype=np.int32)
D = np.array(1, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

S shape: (), X=10
L shape: (), X=0
D shape: (), X=1
Output shape: (0,), Result = []


In [6]:
# Case N4 - inverted bounds and negative delta
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(10, dtype=np.int32)
L = np.array(0,  dtype=np.int32)
D = np.array(-1, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

S shape: (), X=10
L shape: (), X=0
D shape: (), X=-1
Output shape: (10,), Result = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


In [7]:
# Case N5 - S equals L
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(3, dtype=np.int32)
L = np.array(3,  dtype=np.int32)
D = np.array(1, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

S shape: (), X=3
L shape: (), X=3
D shape: (), X=1
Output shape: (0,), Result = []


In [8]:
# This cell MUST FAIL!
# Case N5 - D equals zero
# Input tensors: S, L, D (scalar tensors)
onnx_type = onnx.TensorProto.INT32
S = np.array(3, dtype=np.int32)
L = np.array(3,  dtype=np.int32)
D = np.array(0, dtype=np.int32)
n_values = num_elements(S, L, D)
session = create_reshape_model([n_values], onnx_type)
do_range(S, L, D, session)

ValueError: Step (D) cannot be zero.